In [35]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GOOGLE_API_KEY']=os.getenv("GEMINI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [36]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [37]:
## Data Ingestion

from langchain_community.document_loaders import WebBaseLoader

In [38]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Pitcher_plant")
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Pitcher_plant', 'title': 'Pitcher plant - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nPitcher plant - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(To

In [39]:
### Load Data --> Docs --> Divide out Text into chunks --> vectorize the chunks --> Vector Embeddings --> Store in VectorDB

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

documents = text_splitter.split_documents(docs)
print(documents)

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Pitcher_plant', 'title': 'Pitcher plant - Wikipedia', 'language': 'en'}, page_content='Pitcher plant - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n

In [40]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="gemma:2b")


In [41]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents, embeddings)

In [42]:
query = "what is the food of pitcher plant?"
result = vectorstore.similarity_search(query)
result[0].page_content

'Digestion[edit]\nWhatever the mechanism of digestion, the prey items are converted into a solution of amino acids, peptides, phosphates, ammonium and urea, from which the plant obtains its mineral nutrition (particularly nitrogen and phosphorus). Like all carnivorous plants, pitcher plants all grow in locations where the soil is too poor in minerals and/or too acidic for most plants to survive. Pitcher plants supplement available nutrients and minerals (which plants normally obtain through their roots) with the constituents of their insect prey.[citation needed]'

In [43]:
##Retrieval Chain, Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """ Answer the following question based on the provided context:
    <context>
    {context}
    </context>
    """
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Answer the following question based on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000026CF1811040>, default_metadata=(), model_kwargs={})
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [44]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"what is the food of pitcher plant?",
})

KeyError: 'context'

In [ ]:
vectorstore

In [ ]:
retreiver = vectorstore.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retreiver, document_chain)

In [ ]:
response = retrieval_chain.invoke({

    "input":"what is plant size"
})

response

{'input': 'what is plant size',
 'context': [Document(id='cdae9ec9-4615-4038-8623-911602b211fb', metadata={'source': 'https://en.wikipedia.org/wiki/Pitcher_plant', 'title': 'Pitcher plant - Wikipedia', 'language': 'en'}, page_content='The rim of the pitcher (peristome) is slippery when moistened by condensation or nectar, causing insects to fall into the trap. The walls of the pitfall may be covered with waxy scales, protruding aldehyde crystals, cuticular folds, downward-pointing hairs, or guard-cell-originating lunate cells, to help prevent escape.[1] The small bodies of liquid contained within the pitcher traps are called phytotelmata. They drown the insect, whose body is gradually dissolved. This may occur by bacterial action (the bacteria being washed into the pitcher by rainfall), or by digestive enzymes secreted by the plant itself. Pitcher trap fluids largely vary in their viscoelasticity and acidity, which then dictates which type of prey they can target. For example, increase